In [1]:
import netCDF4
import pyproj
import numpy as np
import pandas as pd

In [2]:
ds = netCDF4.Dataset('/Users/baart_f/data/odv/data_from_SDN_2017-11_TS_profiles_non-restricted_med.nc')

In [3]:
station_lat = ds.variables['latitude'][:]
station_lon = ds.variables['longitude'][:]
lon_i = 10
lat_i = 30
lon = np.zeros_like(station_lon) + lon_i
lat = np.zeros_like(station_lat) + lat_i

In [4]:
wgs84 = pyproj.Geod(ellps='WGS84')


In [5]:
_, _, distance = wgs84.inv(
    lon, lat,
    station_lon, station_lat
)


In [6]:
idx = distance.argmin()
idx

389391

In [18]:
var = ds.variables['metavar1']
var.dtype == 'S1'

True

In [8]:
cruise = netCDF4.chartostring(ds.variables['metavar1'][idx])
profile = netCDF4.chartostring(ds.variables['metavar2'][idx])
type_ = ds.variables['metavar3'][idx]
cdi_id = netCDF4.chartostring(ds.variables['metavar4'][idx])
edmo_code = ds.variables['metavar5'][idx]
profile, cdi_id, edmo_code

(array('90460', dtype='<U75'),
 array('FS35196900000_90460_H13', dtype='<U36'),
 486)

In [9]:
cdi_ids = netCDF4.chartostring(ds.variables['metavar4'][:])

In [10]:
%%timeit
np.argmax(cdi_ids == 'FS35196900000_90460_H13')

10.2 ms ± 543 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
var_names = [
    name 
    for name, var 
    in ds.variables.items() 
    if name.startswith('var') and not '_' in name
]

# add the variables to the list
variables = {}
for var_name in var_names:
    var = ds.variables[var_name]
    variables[var.long_name] = var[idx]

df = pd.DataFrame(data=variables)
# get rid of missing data
df = df.dropna(how='all')

# get metadata
date_nums = ds.variables['date_time'][idx]
date_units = ds.variables['date_time'].units
date = netCDF4.num2date(date_nums, date_units)
records = df.to_json(orient='records')
response = {
    "series": records,
    "meta": {
        "date": date
    }
}
response

{'series': '[{"Depth":0.0,"ITS-90 water temperature":22.7000007629,"Water body salinity":null},{"Depth":5.0,"ITS-90 water temperature":21.7999992371,"Water body salinity":null},{"Depth":10.0,"ITS-90 water temperature":21.7000007629,"Water body salinity":null},{"Depth":30.0,"ITS-90 water temperature":21.7000007629,"Water body salinity":null}]',
 'meta': {'date': datetime.datetime(1969, 11, 9, 13, 0)}}

In [12]:
date_nums.shape
ds.variables['var2'].shape

(451610, 9816)

In [13]:
ds.variables['var1']

<class 'netCDF4._netCDF4.Variable'>
float32 var1(N_STATIONS, N_SAMPLES)
    long_name: Depth
    units: m
    comment: Codes: SDN:P01::ADEPZZ01 SDN:P06::ULAA
    ancillary_variables: var1_qc var1_err
    C_format: %.2f
    FORTRAN_format: F12.2
    _FillValue: -10000000000.0
unlimited dimensions: 
current shape = (451610, 9816)
filling on